In [ ]:
import math
from Bio import SeqIO
import pickle as pkl
from cutadapt import adapters, modifiers
# from dnaio._core import SequenceRecord

In [ ]:
# Quality score cutoff default = 30
phred_cutoff = 30
# 3'-adaptor: MADAP, 4N or Normal
adaptor_3 = "MADAP" # or "Normal"
# 5'-adaptor = 4N for N4_List_Unique or Normal for TCTA
adaptor_5 = "4N" # 4N or "Normal"

log = {"Quality Score Cutoff": phred_cutoff, "3'-Adaptor Sequence": adaptor_3, "5'-Adaptor": adaptor_5}

In [ ]:
# def get_folder_names(directory):
#     experiment_name = directory.split(sep="/")[-1]

#     # Fastq files use the "-" instead of the "_"
#     fastq_readable_name = experiment_name.replace("_", "-").upper()

#     nice_name = fastq_readable_name[0].upper() + fastq_readable_name[1:].lower()
#     nice_name = nice_name.replace("-", "_")

#     log.update(
#         {"Folder_Name": directory,
#          "Corrected_Name": fastq_readable_name,
#          "Nice_Name": nice_name}
#     )

#     return experiment_name, fastq_readable_name, nice_name

# Folder_Name, Corrected_Name, Nice_Name = get_folder_names(os.getcwd())

In [30]:
def get_short_RNA_strands(vcg_file_path):
    # Read lines from the file and remove starting 'p' characters, newline characters, and blank lines.
    with open(vcg_file_path, "r") as file:
        return [line[1:].strip() for line in file.readlines() if line.strip()]

def RNA_list_to_DNA(RNAs):
    return [str(Seq(seq).back_transcribe()) for seq in RNAs]


vcg_file_path = "../Ext_50\VCG_List.txt"
VCG_DNA_list = RNA_list_to_DNA(get_short_RNA_strands(vcg_file_path))
VCG_DNA_list

['AC',
 'AT',
 'CA',
 'CC',
 'CG',
 'GA',
 'GC',
 'GG',
 'GT',
 'TC',
 'TG',
 'ACA',
 'ACC',
 'ACG',
 'ATC',
 'ATG',
 'CAC',
 'CAT',
 'CCA',
 'CGC',
 'CGT',
 'GAT',
 'GCA',
 'GCG',
 'GGT',
 'GTG',
 'TCA',
 'TGA',
 'TGC',
 'TGG',
 'TGT',
 'ACAC',
 'ACCA',
 'ACGC',
 'ATCA',
 'ATGC',
 'CACA',
 'CACC',
 'CACG',
 'CATC',
 'CCAC',
 'CGCA',
 'CGTG',
 'GATG',
 'GCAT',
 'GCGT',
 'GGTG',
 'GTGA',
 'GTGG',
 'GTGT',
 'TCAC',
 'TGAT',
 'TGCG',
 'TGGT',
 'TGTG',
 'ACACG',
 'ACCAC',
 'ACGCA',
 'ATCAC',
 'ATGCG',
 'CACAC',
 'CACCA',
 'CACGC',
 'CATCA',
 'CCACA',
 'CGCAT',
 'CGTGT',
 'GATGC',
 'GCATC',
 'GCGTG',
 'GGTGA',
 'GTGAT',
 'GTGGT',
 'GTGTG',
 'TCACC',
 'TGATG',
 'TGCGT',
 'TGGTG',
 'TGTGG',
 'ACACGC',
 'ACCACA',
 'ACGCAT',
 'ATCACC',
 'ATGCGT',
 'CACACG',
 'CACCAC',
 'CACGCA',
 'CATCAC',
 'CCACAC',
 'CGCATC',
 'CGTGTG',
 'GATGCG',
 'GCATCA',
 'GCGTGT',
 'GGTGAT',
 'GTGATG',
 'GTGGTG',
 'GTGTGG',
 'TCACCA',
 'TGATGC',
 'TGCGTG',
 'TGGTGA',
 'TGTGGT',
 'ACACGCA',
 'ACCACAC',
 'ACGCATC',
 'ATCAC

In [31]:
# Saving VCG_DNA_list to a pickle for easier access
with open("VCG_DNA_list.pkl", "wb") as file:
    pickle.dump(VCG_DNA_list, file)

In [ ]:
def five_prime_motif(adaptor_5):
    '''
    Depending on what kind of adaptor was used in the sequencing run,
    we will look for different short sequences (aka motifs) in order
    to identify the adaptor.

    Args:
        adaptor_5 (str): The kind of adaptor we are using on the 5' end.

    Returns:
        What sequence of nucleotides we should look for to identify our adaptor.
    '''

    all_unique_N4s = [''.join(N4) for N4 in product("ACTG", repeat=4)]

    if adaptor_5 == "4N":
        log.update({"5'-adaptor: ": adaptor_5, "N4 combinations": len(all_unique_N4s)})
        return all_unique_N4s

    elif adaptor_5 == "Normal":
        log.update({"5'-Adaptor: ": adaptor_5, "5'-Adaptor Sequence": motif})
        return "TCTA"

In [ ]:
def average_phred(scores):
    '''
    This function takes a list of Phred quality scores from a read,
    converts them into error probabilities and finds the mean,
    then converts this average error probability back into an overall
    Phred quality score for that read.

    Args:
        scores (list[int]): List of quality Phred scores for each nucleotide from
                            a sequenced read.

    Returns:
        average_score (int): Average Phred quality score for the entire read
    '''

    num_records = len(scores)
    add_p = 0

    # Converting the Phred quality scores into error probabilities
    for i in scores:
        p = 10 ** (-i / 10)
        add_p = add_p + p

    p_bar   = add_p / num_records
    return -10 * math.log10(p_bar)

def filter_reads(read_file, output_directory, output_name):
    '''
    This function takes in a file of reads and sorts them into those
    above and below the Phred cutoff score. Two files are created, one for
    for those below the cutoff, and those above the cutoff. These files are
    saved into the output_directory.

    Args:
        read_file (str): Fastq file containing the sequenced reads
        output_directory (str): Destination of the output files
        output_name: Part of the output names that are shared

    Returns:
        low_qual (list): Reads with an average Phred score below the cutoff
        high_qual (list): Reads with an average Phred score above the cutoff
    '''

    low_qual = []
    high_qual = []
    total_reads = 0

    for record in SeqIO.parse(f"{read_file}", "fastq"):
        total_reads += 1
        scores = record.letter_annotations["phred_quality"]

        if average_phred(scores) < phred_cutoff:
            low_qual.append(record)
        else:
            high_qual.append(record)

    SeqIO.write(low_qual, f"{output_directory}/low_quality_{output_name}.fastq", "fastq") # save low quality reads to file
    SeqIO.write(high_qual, f"{output_directory}/high_quality_{output_name}.fastq", "fastq") # save high quality reads to file

    percentage_low_qual = (len(low_qual) / total_reads) * 100
    percentage_high_qual = (len(high_qual) / total_reads) * 100

    print(f"Number of total reads: {total_reads}")
    print(f"Number of Low Quality reads written out: {len(low_qual)} ({percentage_low_qual:.2f}%)")
    print(f"Number of High Quality reads written out: {len(high_qual)} ({percentage_high_qual:.2f}%)")

    log.update({f"Number of total reads:": str(total_reads),
                f"Number of Low Quality reads written out:": (str(len(low_qual)) + f" ({percentage_low_qual:.2f}%)"),
                f"Number of High Quality reads written out:": (str(len(high_qual)) + f" ({percentage_high_qual:.2f}%)")})

    return low_qual, high_qual

In [ ]:
# This takes a while
# Filtering all input reads for high quality reads, low quality reads are
directory = "/work/"
read_file = "EXT-50_S3_L001_R1_001.fastq"

filter_reads(directory + read_file, directory, "test_output")

# Creating df_4N on the fly (slower)

In [ ]:
# https://www.bioinformatics.org/sms/iupac.html

'''
Find adapters using cutadapt.
'''


seq_3p = "AGATCGGAAGAGCACACGTCT"
errors = 0
adaptor_3p = adapters.FrontAdapter(sequence=seq_3p,
                                    max_errors=errors,
                                    adapter_wildcards=True,
                                    indels=False)

# seq = "TCTAGTGAATCAGATCGGTTCCGACTAGATCGGAAGAGCACACGTCTGAACTCCAGTCACAGTCAACAATCTC"
# read = SequenceRecord(
#         name = "test_seq",
#         sequence = seq)

def format_seq(seq: str, adaptor_3p):
    matched_seq = adaptor_3p.match_to(seq)

    if matched_seq:
        num_errors = matched_seq.errors
        adaptor_start = matched_seq.rstart
        return seq[0:4] + "_" + seq[4:adaptor_start] + "-" + seq[adaptor_start:], num_errors
    return None

high_quality_reads_file = "..\Ext_50\high_quality_Ext_50.fastq"
# format_seq(seq, adaptor_3p) # for troubleshooting

formatted_seqs = []
for record in SeqIO.parse(high_quality_reads_file, "fastq"):
    formatted = format_seq(str(record.seq), adaptor_3p)

    if formatted:
        formatted_seqs.append(formatted)

In [ ]:
seqs = []
errs = []
for seq, err in formatted_seqs:
    seqs.append(seq)
    errs.append(err)

df_4N = pd.DataFrame({"Seq": seqs, "Errors":errs})
df_4N

# Loading df_4N from file (quicker)

Using Pandas vectorization methods instead of .apply() to edit our DataFrame because it is generally much more effeciant on large datasets.

In [ ]:
df_4N = pd.read_csv("..\Ext_50\_deepnote_work_4N.csv.gz")

df_4N["5_Adap"] = df_4N["Seq"].str[:4]
df_4N["Less_3"] = df_4N["Seq"].str.split("-", n=1).str[0]

df_4N["Spacer"]     = df_4N["Less_3"].str[-2:]
df_4N["Rand_4"]     = df_4N["Less_3"].str[-6:-2]
df_4N["MADAP_Code"] = df_4N["Less_3"].str[-9:-6]
df_4N["Bind_6"]     = df_4N["Less_3"].str[-15:-9]
df_4N["4N"]         = df_4N["Less_3"].str[-19:-15]
df_4N["Pre_VCG"]    = df_4N["Less_3"].str[:-19]

df_4N["VCG"]     = df_4N["Pre_VCG"].str[5:]
df_4N["VCG_Len"] = df_4N["VCG"].str.len()
df_4N["3_Adap"]  = df_4N["Seq"].str[5:].str.split("-", n=1).str[1].str[:21]
df_4N["UMI"]     = df_4N["4N"] + df_4N["Rand_4"]

log.update({"No. of reads with adaptors":df_4N.shape[0]})

# Reordering the dataframe for me intuitive reading
df_4N = df_4N[["Seq", "Errors", "5_Adap", "VCG", "VCG_Len", "4N", "Bind_6", "MADAP_Code", "Rand_4", "Spacer", "3_Adap", "UMI"]]
df_4N

In [ ]:
df_4N_reduced = df_4N.query("VCG_Len != 0")
log.update({"No. of reads with 'VCG' Length > 0":df_4N_reduced.shape[0]})
df_4N_reduced

In [ ]:
df_4N_VCG = df_4N_reduced.query("VCG in @VCG_DNA_list")
log.update({"No. of reads in VCG":df_4N_VCG.shape[0]})
df_4N_VCG

## Saving our three dataframes (4N, VCG, and reduced) to pkl files so they can quickly and easily be accessed in other files

In [19]:
# Only need to run this once

with open("df_4N.pkl", "wb") as file_1:
    pkl.dump(df_4N, file_1)

with open("df_4N_reduced.pkl", "wb") as file_2:
    pkl.dump(df_4N_reduced, file_2)

with open("df_4N_VCG.pkl", "wb") as file_3:
    pkl.dump(df_4N_VCG, file_1)

ValueError: write to closed file